In [ ]:
# Download the dataset
file_id = '1TLPrez6to0VRQCxt5L4NgK4tejMBYbTA'

output_file = 'Dataset.zip'

# Generate the download link
download_link = f'https://drive.google.com/uc?id={file_id}'

# Download the file
!gdown $download_link -O $output_file

In [ ]:
!unzip /content/Dataset.zip

In [ ]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from keras.utils import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = "/content/Dataset/train/"
validation_dir = "/content/Dataset/validation/"

sorted_labels = [
    'ayam', 'bawang bombay', 'bawang merah', 'bawang putih', 'bayam', 'brokoli',
    'buncis', 'cabai', 'daun singkong', 'ikan', 'jagung',
    'jamur',  'kacang panjang', 'kambing', 'kangkung', 'kentang', 'kol', 'mie', 'nasi',
    'sapi', 'sawi', 'serai',  'tahu', 'taoge', 'telur', 'tempe', 'terong', 'timun',
    'tomat',  'udang', 'wortel'
]

# train directories
train_directories = {label: os.path.join(train_dir, label) for label in sorted_labels}

# validation directories
validation_directories = {label: os.path.join(validation_dir, label) for label in sorted_labels}

for label in sorted_labels:
    print('Total training', label, 'images:', len(os.listdir(train_directories[label])))

for label in sorted_labels:
    print('Total validation', label, 'images:', len(os.listdir(validation_directories[label])))

jamur_file = os.listdir(train_directories['jamur'])
print(jamur_file[:5])

In [ ]:
# Feature extractor layer
feature_extractor_url = 'https://tfhub.dev/google/efficientnet/b7/feature-vector/1'
feature_extractor_layer = hub.KerasLayer(feature_extractor_url, input_shape=[512,512,3])
feature_extractor_layer.trainable = False

In [ ]:
# Model
model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dense(1024, activation="relu"),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(31, activation="softmax")
])

model.summary()

In [ ]:
model.compile(optimizer= 'Adam',
              loss= 'categorical_crossentropy',
              metrics= ['accuracy'])

In [ ]:
# Add data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale= 1./255,
                                   rotation_range= 50,
                                   width_shift_range= 0.3,
                                   height_shift_range= 0.3,
                                   shear_range= 0.3,
                                   zoom_range= 0.3,
                                   horizontal_flip= True,
                                   fill_mode= 'nearest')

# Flow training images in batches of 31 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size= 31,
                                                    class_mode= 'categorical',
                                                    target_size= (512, 512))

validation_datagen = ImageDataGenerator(rescale = 1./255)

# Flow validation images in batches of 31 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                    batch_size= 31,
                                                    class_mode= 'categorical',
                                                    target_size= (512, 512))

In [ ]:
class Callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = Callback()

# Train the model
history = model.fit(train_generator,
                    epochs= 20,
                    validation_data= validation_generator,
                    verbose= 1,
                    callbacks= callbacks)

In [ ]:
# Plot the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

# Plot the accuracy
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

# Plot the loss
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
# Predictions from validation folder
from keras.preprocessing import image

for label in sorted_labels:
  folder = validation_directories[label]

  for path_file in os.listdir(folder):
    # Get value from image
    path_file = os.path.join(folder, path_file)
    img = image.load_img(path_file, target_size=(512, 512))
    x = img_to_array(img)
    x = x/255
    x = np.expand_dims(x, axis=0)

    # Predict the image label
    images = np.vstack([x])
    classes = model.predict(images, batch_size=31)

    # Take the class index with the highest probability
    predicted_class_index = classes.argmax()

    # Convert class indices to label
    predicted_label = sorted_labels[predicted_class_index]

    print("Label:", label, "Predicted Label:", predicted_label)

In [ ]:
# Test the model using the uploaded image
from google.colab import files
from PIL import Image as PILImage

uploaded = files.upload()

for fn in uploaded.keys():
  # Get value from image
  path = '/content/' + fn
  img = PILImage.open(path)
  img = image.load_img(path, target_size=(512, 512))

  # Display image
  plt.imshow(img)
  plt.axis('off')
  plt.show()

  x = img_to_array(img)
  x = x/255
  x = np.expand_dims(x, axis=0)

  # Predict the image label
  images = np.vstack([x])
  classes = model.predict(images, batch_size=31)

  # Sort the classes from highest probability
  sorted_classes = np.argsort(classes)[0, ::-1]

  # Print the detected ingredients with a threshold of 0.05
  print("Ingredients detected:")
  for index in sorted_classes:
    label = sorted_labels[index]
    prob = classes[0, index]
    if prob > 0.05:
      print(f"{label}: {prob:.4f}")

  # Delete images after use
  os.remove(path)

In [ ]:
# Save the model
model.save("Model.h5")

In [ ]:
# Move the saved model to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

import shutil

local_file_path = '/content/Model.h5'
drive_path = '/content/gdrive/My Drive/'

shutil.copy(local_file_path, drive_path)